In [2]:
from transformers import ElectraModel, ElectraTokenizerFast
from sklearn.metrics.pairwise import cosine_similarity
from model import modelWithPooling
import pandas as pd
import numpy as np
import torch


model = ElectraModel.from_pretrained("../model/disc_book_final")
tokenizer = ElectraTokenizerFast.from_pretrained("monologg/koelectra-base-v3-discriminator")

model_with_pooling = modelWithPooling(model, tokenizer).to('mps')


In [2]:
data =  pd.read_csv('../data/bookList/pre_book_total_128.csv',index_col=0)['sen'].values

In [ ]:
from datasets import Dataset
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
dataset = Dataset.from_pandas(pd.DataFrame(data, columns=['sen']))

def tokenizing_function(batch) :
    lst = []
    for i in batch :
        lst.append(i['sen'])

    return tokenizer(lst,truncation=True,padding=True,return_tensors='pt')
    

dataloader = DataLoader(dataset.select(range(32)),16,collate_fn=tokenizing_function)

model_with_pooling.eval()

lst = []
for num, item in enumerate(tqdm(dataloader)) :
    item.to('mps')
    x = model_with_pooling(**item)['sentence_embedding'].detach()
    lst.append(x)
    if num % 1000 == 0 and num != 0 :    
        torch.save(torch.cat(lst),f'save/embedding_{num}.pt')
        lst.clear()

### Doc Extraction

In [12]:
import utils
from transformers import ElectraModel, ElectraTokenizerFast
from sklearn.metrics.pairwise import cosine_similarity
from model import modelWithPooling
import pandas as pd
import numpy as np
import torch

device='mps'
model = ElectraModel.from_pretrained("../model/bi_encoder_finish")
tokenizer = ElectraTokenizerFast.from_pretrained("monologg/koelectra-base-v3-discriminator")

model_with_pooling = modelWithPooling(model, tokenizer).to(device)

raw_data = pd.read_csv("../data/bookList/raw_book_info_list.csv", index_col=0)

englist= pd.read_csv('../data/preprocess/englist.csv')

raw_data.head(2)

,book_title,book_toc,book_intro,publisher
0,한 권으로 끝내는 메타버스 크리에이터,"['메타버스란', '왜 메타버스인가', '메타버스의 유형을 알아보자', '메타버스 ...",[],[]
1,Do it! 점프 투 파이썬: 라이브러리 예제 편,"['', '텍스트 다루기', '문자열을 줄여 표시하려면 textwrap shorte...",['이 책은 Do it 점프 투 파이썬 의 박응용 저자가 그동안 수많은 독자에게 받...,['실무에서 자주 쓰는 파이썬 라이브러리는 다 있다 필수 파이썬 라이브러리 개 엄선...


In [2]:
def tokenizing_function(text,max_length=128):
    token = tokenizer(
        text,
        truncation=True,
        padding=True,
        max_length=max_length,
        stride=20,
        return_overflowing_tokens=True,
        return_tensors="pt",
    )
    token.pop("overflow_to_sample_mapping")
    return token

In [37]:
from ast import literal_eval
from datasets import Dataset
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import utils
import re

dataset = Dataset.from_pandas(raw_data.reset_index(drop=True))
dataloader = DataLoader(dataset.select(range(3,4)),batch_size=1)

lst = []
for n, x in enumerate(tqdm(dataloader)) :
    # 제목 
    title = x.pop("book_title")[0]

    x = [literal_eval(v[0]) for k,v in x.items()]
    x = [v for j in x for v in j ]

    x = list(filter(None, x))
    x = re.sub(r"[^\w\s]", "", " ".join(x))
    doc = utils.trans_eng_to_han(x, englist=englist)
    doc = ' '.join(doc)


    # 전체 Embedding
    token = tokenizing_function(doc).to(device)
    logits = model_with_pooling(**token)["sentence_embedding"]
    logits = logits.detach()

    # 대표 Embedding
    doc_embedding = torch.sum(logits,dim=0)/logits.size(0)
    doc_embedding = doc_embedding.unsqueeze(0)
    
    lst.append(dict(title=title,rep_embedding=doc_embedding))

    if n % 1000 == 0 and n != 0 :    
        torch.save(lst,f'save/bi_encoder_doc_{int(n/1000)}.pt')
        lst.clear()

  0%|          | 0/1 [00:00<?, ?it/s]

In [39]:
torch.sum(lst[0]['rep_embedding'])

tensor(-14.9516, device='mps:0')